In [4]:
import os 
import json

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from termcolor import colored

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten

from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import Mean, SparseCategoricalAccuracy

from tensorflow.keras.layers import InputLayer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Flatten, Dense, Activation

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [8]:
model = Sequential()
#------CONV_1---------
model.add(Conv2D(filters = 10, kernel_size = (3,3), 
                 padding = "valid", name = "conv_1"))
model.add(MaxPooling2D(pool_size = (2,2), strides = 2, 
                       name = "conv_1_maxpool"))
model.add(Activation("relu", 
                     name = "conv_1_act"))
#------CONV_2---------
model.add(Conv2D(filters = 10, kernel_size = (3,3), 
                 padding = "valid", name = "conv_2"))
model.add(MaxPooling2D(pool_size = (2,2), strides = 2, 
                       name = "conv_2_maxpool"))
model.add(Activation("relu", 
                     name = "conv_2_act"))
#------MLP(FC)---------
model.add(Flatten())
model.add(Dense(units = 32, activation = "relu", name = "dense_1"))
model.add(Dense(units = 10, activation = "softmax", name = "dense_2"))

#------build---------
model.build(input_shape = (None,28,28,1))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 26, 26, 10)        100       
_________________________________________________________________
conv_1_maxpool (MaxPooling2D (None, 13, 13, 10)        0         
_________________________________________________________________
conv_1_act (Activation)      (None, 13, 13, 10)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 11, 11, 10)        910       
_________________________________________________________________
conv_2_maxpool (MaxPooling2D (None, 5, 5, 10)          0         
_________________________________________________________________
conv_2_act (Activation)      (None, 5, 5, 10)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 250)              

In [22]:
#print(colored("","blue"),"\n",,"\n")
#List containing layer object of the model
print(colored("model.layers","blue"),"\n",model.layers,"\n")
print(colored("len(model.layers)","blue"),"\n",len(model.layers),"\n")

model.layers 
 [<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7faf2e80c940>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7faf2e8d4100>, <tensorflow.python.keras.layers.core.Activation object at 0x7faf2edeabe0>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7faf2e8e4190>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7faf2edff6a0>, <tensorflow.python.keras.layers.core.Activation object at 0x7faf2edfaa60>, <tensorflow.python.keras.layers.core.Flatten object at 0x7faf2edff700>, <tensorflow.python.keras.layers.core.Dense object at 0x7faf2ede3ac0>, <tensorflow.python.keras.layers.core.Dense object at 0x7faf2ede3370>] 

len(model.layers) 
 9 



In [38]:
#get_config, get_weights
first_layer = model.layers[0]
first_layer_config = first_layer.get_config()

#use json to printy nested dic
print(json.dumps(first_layer_config, indent = 3))

{
   "name": "conv_1",
   "trainable": true,
   "dtype": "float32",
   "filters": 10,
   "kernel_size": [
      3,
      3
   ],
   "strides": [
      1,
      1
   ],
   "padding": "valid",
   "data_format": "channels_last",
   "dilation_rate": [
      1,
      1
   ],
   "groups": 1,
   "activation": "linear",
   "use_bias": true,
   "kernel_initializer": {
      "class_name": "GlorotUniform",
      "config": {
         "seed": null
      }
   },
   "bias_initializer": {
      "class_name": "Zeros",
      "config": {}
   },
   "kernel_regularizer": null,
   "bias_regularizer": null,
   "activity_regularizer": null,
   "kernel_constraint": null,
   "bias_constraint": null
}


In [47]:
for layer in model.layers:
    layer_config = layer.get_config()
    layer_name = layer_config["name"]
    if layer_name.startswith("conv") and len(layer_name.split("_")) <= 2:
        print(colored("Layer Name:","blue"),layer_name)
        print("Num Filter:", layer_config["filters"])
        print("Kernel Size:", layer_config["kernel_size"])
        print("Padding:", layer_config["padding"])
        print()


Layer Name: conv_1
Num Filter: 10
Kernel Size: (3, 3)
Padding: valid

Layer Name: conv_2
Num Filter: 10
Kernel Size: (3, 3)
Padding: valid



In [61]:
final_layer = model.layers[-1]
print(colored("type(final_layer.get_weights())","blue"),"\n",type(final_layer.get_weights()),"\n")

#weights
print(colored("type(final_layer.get_weights()[0])","blue"),"\n",type(final_layer.get_weights()[0]),"\n")
print(colored("final_layer.get_weights()[0]).shape","blue"),"\n",final_layer.get_weights()[0].shape,"\n")

#bias
print(colored("type(final_layer.get_weights()[1])","blue"),"\n",type(final_layer.get_weights()[1]),"\n")
print(colored("final_layer.get_weights()[1]).shape","blue"),"\n",final_layer.get_weights()[1].shape,"\n")

#bais is initialized with zeros
display(final_layer.get_weights()[1])

type(final_layer.get_weights()) 
 <class 'list'> 

type(final_layer.get_weights()[0]) 
 <class 'numpy.ndarray'> 

final_layer.get_weights()[0]).shape 
 (32, 10) 

type(final_layer.get_weights()[1]) 
 <class 'numpy.ndarray'> 

final_layer.get_weights()[1]).shape 
 (10,) 



array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)